In [55]:
import json
import os
import numpy as np
import pandas as pd
import tensorflow as tf


from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from src.data_processing.pipelines.ClassifierPipe import ClassifierPipe
from src.utilities.os_helpers import set_up_directories
from src.data_processing.processors.TrainingProcessor import TrainingProcessor
from src.models.BaseClassifier import BaseClassifier

ImportError: cannot import name 'tfa' from 'tensorflow' (/Users/mds8301/anaconda3/envs/enigma/lib/python3.11/site-packages/tensorflow/__init__.py)

In [59]:

DATA_PATH = "/Users/mds8301/iterm_data_storage/raw_data_raw_data.parquet.gzip"
MAIN_DIR = "/Users/mds8301/iterm_data_storage"
EXPERIMENT_NAME = "base_classifier_tuning"

# path to experiment directory
EXPERIMENT_DIR = os.path.join(MAIN_DIR, EXPERIMENT_NAME)
set_up_directories(EXPERIMENT_DIR)

processor_pipe = (ClassifierPipe(DATA_PATH)
                  .read_raw_data()
                  .calculate_max_min_signal()
                  .split_data(test_size=0.3,
                              test_dev_size=0.5,
                              split_group="mouse_id",
                              stratify_group="sex",
                              target='action',
                              save_subject_ids=True,
                              path_to_save=os.path.dirname(DATA_PATH))
                  .transorm_data(numeric_target_dict={'avoid': 1, 'escape': 0})
                  )


space = {
    'number_of_layers': hp.choice('number_of_layers', [2,3]),
    'number_of_units': hp.choice('number_of_units', [5, 10]),
    "dropout_rate": hp.choice('dropout_rate', [0.1, 0.2]),
    "learning_rate": hp.choice('learning_rate', [0.0001, 0.1]),
    "batch_size": hp.choice('batch_size', [32, 64]),
    "epochs": hp.choice('epochs', [1,2,3]),
    "optimizers": hp.choice('optimizers', ['adam', 'sgd'])

}

trials = Trials()


def objective(params):

    number_of_layers = params['number_of_layers']
    number_of_units = params['number_of_units']
    dropout_rate = params['dropout_rate']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']
    optimizer = params['optimizers']

    # set up model
    model = BaseClassifier(number_of_layers, number_of_units, dropout_rate)

    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        


    metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc-roc')]

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', metrics=metrics)

    # train model
    model.fit(processor_pipe.X_train,
              processor_pipe.y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(processor_pipe.X_dev, processor_pipe.y_dev)
              )

    evaluation = model.evaluate(processor_pipe.X_test, processor_pipe.y_test)

    def calculate_f1_score(precission, recall):
        f1 = 2 * (precission * recall) / (precission + recall)
        return f1
        

    all_results = {}
    all_results['params'] = params
    for name, value in zip(model.metrics_names, evaluation):
        all_results[name] = value
    # results['model'] = model
    all_results['f1_score'] = calculate_f1_score(evaluation[1], evaluation[2])
    all_results['status'] = STATUS_OK
    evaluation.append(all_results['f1_score'])

    with open(os.path.join(EXPERIMENT_DIR, 'results.json'), 'a+') as f:
        json.dump(all_results, f, indent=1)
    
    f1_score = evaluation[-1]


    return -1 * f1_score


def run_trials():
    best_trials = fmin(objective,
                       space=space,
                       algo=tpe.suggest,
                       max_evals=10,
                       trials=trials)
    best_params = space_eval(space, best_trials)
    
    with open(os.path.join(EXPERIMENT_DIR, 'best_params.json'), 'a+') as f:
        json.dump(best_params, f)

    return best_trials
best_trials = run_trials()


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Epoch 1/2                                             

127/127 [==============================] - 8s 57ms/step - loss: 0.6583 - accuracy: 0.6785 - precision: 0.7107 - recall: 0.7675 - auc-roc: 0.7195 - val_loss: 0.6199 - val_accuracy: 0.6734 - val_precision: 0.5410 - val_recall: 0.7474 - val_auc-roc: 0.7445

Epoch 2/2                                             

127/127 [==============================] - 2s 17ms/step - loss: 0.5937 - accuracy: 0.7118 - precision: 0.7401 - recall: 0.7884 - auc-roc: 0.7502 - val_loss: 0.8841 - val_accuracy: 0.5233 - val_precision: 0.4310 - val_recall: 0.9211 - val_auc-roc: 0.7585

82/82 [==============================] - 1s 15ms/step - loss: 0.5456 - accuracy: 0.7350 - precision: 0.7222 - recall: 0.9222 - auc-roc: 0.8025

 10%|█         | 1/10 [00:11<01:47, 11.89s/trial, best loss: -0.7285728635913843]

Epoch 1/2                                                                        

127/127 [==============================] - 8s 52ms/step - loss: 0.5914 - accuracy: 0.6978 - precision: 0.7319 - recall: 0.7696 - auc-roc: 0.7406 - val_loss: 0.7586 - val_accuracy: 0.5988 - val_precision: 0.4756 - val_recall: 0.8711 - val_auc-roc: 0.7355

Epoch 2/2                                                                        

127/127 [==============================] - 2s 14ms/step - loss: 0.5614 - accuracy: 0.7360 - precision: 0.7626 - recall: 0.8022 - auc-roc: 0.7732 - val_loss: 0.7727 - val_accuracy: 0.6163 - val_precision: 0.4879 - val_recall: 0.8500 - val_auc-roc: 0.7448

82/82 [==============================] - 1s 15ms/step - loss: 0.5171 - accuracy: 0.7678 - precision: 0.7974 - recall: 0.8326 - auc-roc: 0.8115

 20%|██        | 2/10 [00:23<01:35, 11.91s/trial, best loss: -0.7823214306082967]

Epoch 1/2                                                                        

127/127 [==============================] - 8s 50ms/step - loss: 0.6213 - accuracy: 0.6605 - precision: 0.6858 - recall: 0.7834 - auc-roc: 0.6983 - val_loss: 0.7498 - val_accuracy: 0.5620 - val_precision: 0.4523 - val_recall: 0.8974 - val_auc-roc: 0.7214

Epoch 2/2                                                                        

127/127 [==============================] - 2s 18ms/step - loss: 0.5761 - accuracy: 0.7291 - precision: 0.7417 - recall: 0.8298 - auc-roc: 0.7601 - val_loss: 0.7258 - val_accuracy: 0.5930 - val_precision: 0.4722 - val_recall: 0.8947 - val_auc-roc: 0.7366

82/82 [==============================] - 1s 15ms/step - loss: 0.5617 - accuracy: 0.7400 - precision: 0.7568 - recall: 0.8482 - auc-roc: 0.7722

 30%|███       | 3/10 [00:34<01:21, 11.57s/trial, best loss: -0.7823214306082967]

Epoch 1/3                                                                        

64/64 [==============================] - 7s 95ms/step - loss: 0.7148 - accuracy: 0.6316 - precision: 0.6628 - recall: 0.7645 - auc-roc: 0.6492 - val_loss: 0.8866 - val_accuracy: 0.5572 - val_precision: 0.4368 - val_recall: 0.7000 - val_auc-roc: 0.6147

Epoch 2/3                                                                        

64/64 [==============================] - 1s 22ms/step - loss: 0.6855 - accuracy: 0.6400 - precision: 0.6718 - recall: 0.7620 - auc-roc: 0.6636 - val_loss: 0.8468 - val_accuracy: 0.5591 - val_precision: 0.4388 - val_recall: 0.7079 - val_auc-roc: 0.6232

Epoch 3/3                                                                        

64/64 [==============================] - 1s 18ms/step - loss: 0.6634 - accuracy: 0.6489 - precision: 0.6806 - recall: 0.7629 - auc-roc: 0.6761 - val_loss: 0.8168 - val_accuracy: 0.5698 - val_precision: 0.4474 - val_recall: 0.7158 - val_auc-roc: 

...

Epoch 1/3                                                                        

64/64 [==============================] - 7s 94ms/step - loss: 0.7158 - accuracy: 0.4308 - precision: 0.5187 - recall: 0.4868 - auc-roc: 0.3781 - val_loss: 0.7143 - val_accuracy: 0.4748 - val_precision: 0.3823 - val_recall: 0.6921 - val_auc-roc: 0.5235

Epoch 2/3                                                                        

64/64 [==============================] - 1s 23ms/step - loss: 0.7088 - accuracy: 0.4446 - precision: 0.5305 - recall: 0.5094 - auc-roc: 0.4031 - val_loss: 0.7058 - val_accuracy: 0.4903 - val_precision: 0.3907 - val_recall: 0.6868 - val_auc-roc: 0.5487

Epoch 3/3                                                                        

64/64 [==============================] - 1s 23ms/step - loss: 0.7023 - accuracy: 0.4720 - precision: 0.5539 - recall: 0.5395 - auc-roc: 0.4392 - val_loss: 0.6966 - val_accuracy: 0.5165 - val_precision: 0.4066 - val_recall: 0.6816 - val_auc-roc: 

Epoch 1/3                                                                        

127/127 [==============================] - 8s 53ms/step - loss: 0.6890 - accuracy: 0.5598 - precision: 0.6051 - recall: 0.7307 - auc-roc: 0.5408 - val_loss: 0.7766 - val_accuracy: 0.4264 - val_precision: 0.3812 - val_recall: 0.8947 - val_auc-roc: 0.5536

Epoch 2/3                                                                        

127/127 [==============================] - 2s 19ms/step - loss: 0.6435 - accuracy: 0.6210 - precision: 0.6440 - recall: 0.7997 - auc-roc: 0.6583 - val_loss: 0.7576 - val_accuracy: 0.4419 - val_precision: 0.3911 - val_recall: 0.9263 - val_auc-roc: 0.6505

Epoch 3/3                                                                        

127/127 [==============================] - 2s 18ms/step - loss: 0.6145 - accuracy: 0.6736 - precision: 0.6865 - recall: 0.8223 - auc-roc: 0.7173 - val_loss: 0.7480 - val_accuracy: 0.4758 - val_precision: 0.4069 - val_recall: 0.9263 - val_auc

Epoch 1/3                                                                        

127/127 [==============================] - 8s 51ms/step - loss: 0.8312 - accuracy: 0.4792 - precision: 0.5506 - recall: 0.6370 - auc-roc: 0.4005 - val_loss: 0.9542 - val_accuracy: 0.4099 - val_precision: 0.3095 - val_recall: 0.4895 - val_auc-roc: 0.4000

Epoch 2/3                                                                        

127/127 [==============================] - 2s 16ms/step - loss: 0.8281 - accuracy: 0.4794 - precision: 0.5506 - recall: 0.6391 - auc-roc: 0.4014 - val_loss: 0.9506 - val_accuracy: 0.4099 - val_precision: 0.3101 - val_recall: 0.4921 - val_auc-roc: 0.4001

Epoch 3/3                                                                        

127/127 [==============================] - 2s 14ms/step - loss: 0.8251 - accuracy: 0.4809 - precision: 0.5516 - recall: 0.6420 - auc-roc: 0.4026 - val_loss: 0.9472 - val_accuracy: 0.4118 - val_precision: 0.3124 - val_recall: 0.4974 - val_auc

64/64 [==============================] - 6s 77ms/step - loss: 0.9135 - accuracy: 0.3960 - precision: 0.4477 - recall: 0.1020 - auc-roc: 0.4058 - val_loss: 0.7723 - val_accuracy: 0.5688 - val_precision: 0.2628 - val_recall: 0.0947 - val_auc-roc: 0.4313

82/82 [==============================] - 2s 21ms/step - loss: 0.9058 - accuracy: 0.3534 - precision: 0.3569 - recall: 0.0691 - auc-roc: 0.3417

 90%|█████████ | 9/10 [01:45<00:11, 11.23s/trial, best loss: -0.7823214306082967]

Epoch 1/2                                                                        

127/127 [==============================] - 7s 47ms/step - loss: 1.1423 - accuracy: 0.3814 - precision: 0.4757 - recall: 0.4751 - auc-roc: 0.2897 - val_loss: 1.1191 - val_accuracy: 0.3886 - val_precision: 0.2932 - val_recall: 0.4684 - val_auc-roc: 0.3656

Epoch 2/2                                                                        

127/127 [==============================] - 2s 15ms/step - loss: 1.1298 - accuracy: 0.3854 - precision: 0.4792 - recall: 0.4810 - auc-roc: 0.2916 - val_loss: 1.1063 - val_accuracy: 0.3895 - val_precision: 0.2937 - val_recall: 0.4684 - val_auc-roc: 0.3681

82/82 [==============================] - 1s 14ms/step - loss: 1.1993 - accuracy: 0.3271 - precision: 0.4397 - recall: 0.3584 - auc-roc: 0.2379

100%|██████████| 10/10 [01:55<00:00, 11.53s/trial, best loss: -0.7823214306082967]
